<a href="https://colab.research.google.com/github/annanasnas/askqe/blob/main/askQE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import json
import os
from glob import glob

### Step 1: Data Preprocessing

Raw data example

In [41]:
file_path = "/content/askqe/data/processed/en-fr.jsonl"

df = pd.read_json(file_path, lines=True)

df[df["id"] == "CMU_1:6"]


,id,en,fr
0,CMU_1:6,and are you having any of the following sympto...,et avez-vous l’un des symptômes suivants en pl...


In [42]:
df.shape

(971, 3)

Translated data example

In [29]:
file_path = "/content/askqe/data/google_translate/en-fr-gt.jsonl"

df = pd.read_json(file_path, lines=True)

df[df["id"] == "CMU_1:6"]


,id,en,fr,gt_en
0,CMU_1:6,and are you having any of the following sympto...,et avez-vous l’un des symptômes suivants en pl...,Et avez-vous l'un des symptômes suivants avec ...


Xcomet score example

In [30]:
file_path = "/content/askqe/data/xcomet/en-fr-xcomet.jsonl"

df = pd.read_json(file_path, lines=True)

df[df["id"] == "CMU_1:6"]


,id,en,fr,gt_en,xcomet_annotation
0,CMU_1:6,and are you having any of the following sympto...,et avez-vous l’un des symptômes suivants en pl...,Et avez-vous l'un des symptômes suivants avec ...,"{'segment_score': 0.973, 'error_spans': [{'tex..."


In [32]:
df[df["id"] == "CMU_1:6"]["fr"][0]

'et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?'

In [33]:
df[df["id"] == "CMU_1:6"]["gt_en"][0]

"Et avez-vous l'un des symptômes suivants avec votre douleur thoracique"

In [34]:
df[df["id"] == "CMU_1:6"]["xcomet_annotation"][0]

{'segment_score': 0.973,
 'error_spans': [{'text': 'douleur tho',
   'confidence': 0.41378363966941806,
   'severity': 'minor',
   'start': 51,
   'end': 63}]}

### Step 2: ContraTICO

Without OPENAIKEY can't be changable (must to pay)

In [49]:
df1 = pd.read_json("/content/askqe/contratico/en-fr/alteration.jsonl", lines=True)
df2 = pd.read_json("/content/askqe/contratico/en-fr/expansion_impact.jsonl", lines=True)
df3 = pd.read_json("/content/askqe/contratico/en-fr/expansion_noimpact.jsonl", lines=True)
df4 = pd.read_json("/content/askqe/contratico/en-fr/intensifier.jsonl", lines=True)
df5 = pd.read_json("/content/askqe/contratico/en-fr/omission.jsonl", lines=True)
df6 = pd.read_json("/content/askqe/contratico/en-fr/spelling.jsonl", lines=True)
df7 = pd.read_json("/content/askqe/contratico/en-fr/synonym.jsonl", lines=True)
df8 = pd.read_json("/content/askqe/contratico/en-fr/word_order.jsonl", lines=True)

df_all = pd.concat([df1[df1["id"] == "CMU_1:6"],
                    df2[df2["id"] == "CMU_1:6"],
                    df3[df3["id"] == "CMU_1:6"],
                    df4[df4["id"] == "CMU_1:6"],
                    df5[df5["id"] == "CMU_1:6"],
                    df6[df6["id"] == "CMU_1:6"],
                    df7[df7["id"] == "CMU_1:6"],
                    df8[df8["id"] == "CMU_1:6"], ], ignore_index=True)

pd.set_option('display.max_colwidth', None)
df_all[["fr", "perturbation", "pert_fr"]]


,fr,perturbation,pert_fr
0,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,alteration,et avez-vous aucun des symptômes suivants en plus de vos douleurs thoraciques ?
1,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,expansion_impact,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques et de votre toux persistante ?
2,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,expansion_noimpact,et avez-vous l'un des symptômes suivants en plus de vos douleurs thoraciques graves ?
3,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,intensifier,et avez-vous l'un des symptômes suivants en plus de vos fortes douleurs thoraciques ?
4,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,omission,et avez-vous l’un des suivants en plus de vos douleurs thoraciques ?
5,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,spelling,et avez-vous l’un des symptômes suivants en po de vos douleurs thoraciques ?
6,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,synonym,et avez-vous l'un des signes suivants en plus de vos douleurs thoraciques ?
7,et avez-vous l’un des symptômes suivants en plus de vos douleurs thoraciques ?,word_order,"Et en plus de vos douleurs thoraciques, avez-vous l'un des symptômes suivants ?"


### Step 3: Question Generation

#### 3.1 Fact generation

1. Fact extraction: we prompt GPT-4o to extract atomic facts that can be inferred from the source sentence

Problem: there is no open-access of atomic facts generator

2. Entailment classification: we use an off-theshelf NLI classifier to assess the binary entailment relationship (entailed or contradictory) between each extracted fact (as the hypothesis) and the source sentence (as the premise)

Problem: there is no open-access of Entailment classification

Result

In [50]:
file_path = "/content/askqe/QG/code/qg_variants.jsonl"

df = pd.read_json(file_path, lines=True)

df[df["id"] == "CMU_1:6"]


,id,en,atomic_facts,semantic_roles
0,CMU_1:6,and are you having any of the following symptoms with your chest pain,"['You are having chest pain.', 'You may have any of the following symptoms with your chest pain.']","{'Verb': 'having', 'ARG0': 'you', 'ARG1': 'any of the following symptoms', 'COM': 'with your chest pain'}"


#### 3.2 Question generation

Vanilla (without facts)

In [ ]:
vanilla = """Task: You will be given an English sentence. Your goal is to generate a list of relevant questions based on the sentence. Output only the list of questions in Python list format without giving any additional explanation.

*** Example Starts ***
Sentence: It is not yet known whether the severity or level of control of underlying health conditions affects the risk for severe disease associated with COVID-19.
Questions: ["What is not yet known?", "What might affect the risk for severe disease associated with COVID-19?", "What is associated with severe disease?", "What conditions are mentioned in the sentence?", "What is the disease mentioned in the sentence?"]

Sentence: The number of accessory proteins and their function is unique depending on the specific coronavirus.
Questions: ["What is unique depending on the specific coronavirus?", "What is unique about the function of accessory proteins?"]
*** Example Ends ***

Sentence: {{sentence}}
Questions: """

Result

In [58]:
TARGET_ID = "CMU_1:6"
BASE_DIR = "/content/askqe/QG/"

rows = []

for model_dir in os.listdir(BASE_DIR):
    dir_path = os.path.join(BASE_DIR, model_dir)
    if not os.path.isdir(dir_path):
        continue

    jsonl_files = glob(os.path.join(dir_path, "vanilla_*.jsonl"))

    for file_path in jsonl_files:
        with open(file_path, "r") as f:
            for line in f:
                data = json.loads(line)
                if data.get("id") != TARGET_ID:
                    continue
                rows.append({
                    "model": model_dir,
                    "questions": data.get("questions")
                })

df_vanilla = pd.DataFrame(rows)
df_vanilla

,model,questions
0,llama-70b,"[""What symptoms are mentioned in the sentence?"", ""What is associated with chest pain?"", ""What is being asked about chest pain?"", ""What symptoms are being inquired about?""]"
1,gemma-9b,"[""What symptoms are being asked about?"", ""What is the context of the chest pain?""]"
2,llama-8b,"['What symptoms are associated with chest pain?', 'What symptoms are you experiencing?']"
3,yi-9b,"[""What symptoms are being referred to?"", ""Are these symptoms causing chest pain?""]"
4,gemma-27b,"[""What symptoms are associated with chest pain?"", ""What are the symptoms being asked about?""]"


Atomic (with atomic facts)

In [ ]:
nli = """Task: You will be given an English sentence and a list of atomic facts, which are short sentences conveying one piece of information. Your goal is to generate a list of relevant questions based on the sentence. Output the list of questions in Python list format without giving any additional explanation. Do not output as code format (```python```).

*** Example Starts ***
Sentence: It is not yet known whether the severity or level of control of underlying health conditions affects the risk for severe disease associated with COVID-19.
Atomic facts: ['It is not yet known whether the severity of underlying health conditions affects the risk for severe disease associated with COVID-19.', 'It is not yet known whether the level of control of underlying health conditions affects the risk for severe disease associated with COVID-19.']
Questions: ["Is it known whether the severity of underlying health conditions affects the risk for severe disease associated with COVID-19?", "Is it known whether the level of control of underlying health conditions affects the risk for severe disease associated with COVID-19?"]

Sentence: The number of accessory proteins and their function is unique depending on the specific coronavirus.
Atomic facts: ['The number of accessory proteins is unique depending on the specific coronavirus.', 'The function of accessory proteins is unique depending on the specific coronavirus.']
Questions: ["What is unique depending on the specific coronavirus?", "What is unique about the function of accessory proteins?"]
*** Example Ends ***

Sentence: {{sentence}}
Atomic facts: {{atomic_facts}}
Questions: """

Result

In [60]:
TARGET_ID = "CMU_1:6"
BASE_DIR = "/content/askqe/QG/"

rows = []

for model_dir in os.listdir(BASE_DIR):
    dir_path = os.path.join(BASE_DIR, model_dir)
    if not os.path.isdir(dir_path):
        continue

    jsonl_files = glob(os.path.join(dir_path, "atomic_*.jsonl"))

    for file_path in jsonl_files:
        with open(file_path, "r") as f:
            for line in f:
                data = json.loads(line)
                if data.get("id") != TARGET_ID:
                    continue
                rows.append({
                    "model": model_dir,
                    "questions": data.get("questions")
                })

df_atomic = pd.DataFrame(rows)
df_atomic

,model,questions
0,llama-70b,"[""Do you have chest pain?"", ""What symptoms are you experiencing with your chest pain?""]"
1,gemma-9b,"[""What symptoms are associated with chest pain?"", ""What other symptoms might you be experiencing with your chest pain?""]"
2,llama-8b,"['Do you have chest pain?', 'Are you having any of the following symptoms with your chest pain?']"
3,yi-9b,"[""What condition is being referred to?"", ""What symptoms may accompany chest pain?""]"
4,gemma-27b,"[""What symptoms are associated with the chest pain?""]"


Semantic (with dictionary of semantic roles)

In [ ]:
srl = """Task: You will be given an English sentence and a dictionary of semantic roles in the sentence. Your goal is to generate a list of relevant questions based on the sentence. Output the list of questions in Python list format without giving any additional explanation. Do not output as code format (```python```).

*** Example Starts ***
Sentence: It is not yet known whether the severity or level of control of underlying health conditions affects the risk for severe disease associated with COVID-19.
Semantic roles: {'Verb1': {'Verb': 'known', 'ARG1': 'It', 'TMP': 'not yet', 'ARG2': 'whether the severity or level of control of underlying health conditions affects the risk for severe disease associated with COVID-19'}, 'Verb2': {'Verb': 'affects', 'ARG0': 'the severity or level of control of underlying health conditions', 'ARG1': 'the risk for severe disease associated with COVID-19'}}
Questions: ["What is not yet known?","When is it not yet known?","What is being questioned in terms of its effect on the risk for severe disease associated with COVID-19?","What affects the risk for severe disease associated with COVID-19?","What is the risk associated with COVID-19?"]

Sentence: The number of accessory proteins and their function is unique depending on the specific coronavirus.
Semantic roles: {'Verb': 'is', 'ARG1': 'The number of accessory proteins and their function', 'MNR': 'unique', 'TMP': 'depending on the specific coronavirus'}
Questions: ["What is unique depending on the specific coronavirus?", "How is the number of accessory proteins and their function described?", "When is the uniqueness of the number of accessory proteins and their function determined?"]
*** Example Ends ***

Sentence: {{sentence}}
Semantic roles: {{semantic_roles}}
Questions: """

Results

In [63]:
TARGET_ID = "CMU_1:6"
BASE_DIR = "/content/askqe/QG/"

rows = []

for model_dir in os.listdir(BASE_DIR):
    dir_path = os.path.join(BASE_DIR, model_dir)
    if not os.path.isdir(dir_path):
        continue

    jsonl_files = glob(os.path.join(dir_path, "semantic_*.jsonl"))

    for file_path in jsonl_files:
        with open(file_path, "r") as f:
            for line in f:
                data = json.loads(line)
                if data.get("id") != TARGET_ID:
                    continue
                rows.append({
                    "model": model_dir,
                    "questions": data.get("questions")
                })

df_semantic = pd.DataFrame(rows)
df_semantic

,model,questions
0,llama-70b,"[""What symptoms are you having with your chest pain?"", ""Who is having any of the following symptoms?"", ""What are you having with your chest pain?"", ""What is being had by you with your chest pain?"", ""What are the symptoms that you are having with your chest pain?""]"
1,gemma-9b,"[""What symptoms are you experiencing with your chest pain?"", ""Are you experiencing any symptoms along with your chest pain?""]"
2,llama-8b,"['What symptoms are you experiencing with your chest pain?', 'Are you experiencing any symptoms with your chest pain?', 'What is the nature of your chest pain?', 'Are you having any symptoms?']"
3,yi-9b,"[""What symptoms are being referred to with your chest pain?"", ""Are you experiencing any of the mentioned symptoms along with your chest pain?""]"
4,gemma-27b,"[""What symptoms are being experienced with chest pain?"", ""Are there any symptoms accompanying the chest pain?""]"


### Step 4: Question Answering

In [ ]:
qa_prompt = """Task: You will be given an English sentence and a list of relevant questions. Your goal is to generate a list of answers to the questions based on the sentence. Output only the list of answers in Python list format without giving any additional explanation. Do not output as code format (```python```).
